<a href="https://colab.research.google.com/github/taresadere/taresadere/blob/main/freeze_of_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import drive

# Accessing My Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install plot_model

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.image import ImageDataGenerator, load_img
# from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.models import Model
# from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras import backend
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout
from keras.models import Sequential
import matplotlib.pyplot as plt
from glob import glob
import os

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
train_path = "drive/My Drive/train"
valid_path ="drive/My Drive/validation"
test_path = "drive/My Drive/test"
IMAGE_SIZE=(224, 224)
from keras.preprocessing import image
classes= ["normal", "follicular", "intense", "scar", "trichiasis", "corneal opacity"]
def model_predict(img_path, model):
    image = cv2.imread(img_path)
    #image = cv2.resize(image, (224, 224))
    image = cv2.resize(image,   IMAGE_SIZE)
    image = image.astype("float") / 255.0
    #image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    res = model.predict(image)
    label = np.argmax(res)
    print("Label", label)
    labelName = classes[label]
    print("Label name:", labelName)
    return labelName

EPOCHS = 10
RANDOM_SEED = 42

if backend.image_data_format() == 'channels_first':
    INPUT_SHAPE = (3, IMAGE_SIZE[0], IMAGE_SIZE[1])
else:
    INPUT_SHAPE = (IMAGE_SIZE[0], IMAGE_SIZE[1], 3)

print(f'input_shape: {INPUT_SHAPE}')

# useful for getting number of output classes
count_of_classes = len(glob(train_path+'/*'))
print(count_of_classes)

input_shape: (224, 224, 3)
6


In [ ]:
# Prepare train/test using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = tuple(IMAGE_SIZE),
                                                 batch_size = BATCH_SIZE,
                                                 class_mode = 'categorical')

valid_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = tuple(IMAGE_SIZE),
                                            batch_size = BATCH_SIZE,
                                            class_mode = 'categorical')

Found 369 images belonging to 6 classes.
Found 68 images belonging to 6 classes.


In [ ]:
# Target class labels
labels_inv = dict((y,x) for x,y in training_set.class_indices.items())
labels_inv

{0: 'corneal',
 1: 'follicular',
 2: 'intense',
 3: 'normal',
 4: 'scar',
 5: 'trichiasis'}

In [ ]:
vgg16= VGG16(input_shape=INPUT_SHAPE, weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
from tensorflow.keras import Model 
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
x = vgg16.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(0.2)(x)
output  = Dense(units=6, activation='softmax')(x)
model = Model(vgg16.input, output)
model = Model(inputs=vgg16.input, output=prediction)

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss=loss, metrics= ['accuracy'])


In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
lrr = ReduceLROnPlateau(monitor='val_accuracy', 
                        patience=3, 
                        verbose=1, 
                        factor=0.4, 
                        min_lr=0.0001)
history = model.fit_generator(
  training_set,
  validation_data=valid_set,
  epochs=EPOCHS,
  steps_per_epoch=len(training_set),
  validation_steps=len(valid_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
12/12 [==============================] - 257s 20s/step - loss: 1.7999 - accuracy: 0.1789 - val_loss: 1.7168 - val_accuracy: 0.2059
Epoch 2/10
12/12 [==============================] - 223s 19s/step - loss: 1.6213 - accuracy: 0.3415 - val_loss: 1.7419 - val_accuracy: 0.1176
Epoch 3/10
12/12 [==============================] - 225s 19s/step - loss: 1.4777 - accuracy: 0.4119 - val_loss: 1.8364 - val_accuracy: 0.1912
Epoch 4/10
12/12 [==============================] - 225s 19s/step - loss: 1.3523 - accuracy: 0.5393 - val_loss: 1.7492 - val_accuracy: 0.2206
Epoch 5/10
12/12 [==============================] - 222s 19s/step - loss: 1.2661 - accuracy: 0.5718 - val_loss: 1.8656 - val_accuracy: 0.1765
Epoch 6/10
12/12 [==============================] - 224s 19s/step - loss: 1.2223 - accuracy: 0.5339 - val_loss: 1.8479 - val_accuracy: 0.2059
Epoch 7/10
12/12 [==============================] - 225s 19s/step - loss: 1.1686 - accuracy: 0.6016 - val_loss: 1.8315 - val_accuracy: 0.2059
Epoch 

In [ ]:
print('Evaluate on test data:')
results = model.evaluate(test_set, batch_size=128, verbose = 2)
print('Test loss = {0}, Test acc: {1}'.format(results[0], results[1]))

In [ ]:
model.save('C:/Users/Dare/Desktop/TDCN GUI/models/resnet.h5')

In [ ]:
from keras.models import load_model
model = load_model("drive/My Drive/vgg.h5")

# making prediction and classification report

In [ ]:
batch_size=32
y_pred = model.predict(valid_set, batch_size=batch_size)

In [ ]:
from sklearn.metrics import classification_report
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix,roc_curve,auc
from PIL import Image
y_pred=model.predict_generator(generator=valid_set,)
y_pred=np.argmax(y_pred,axis=1)
print('classification report')
print(classification_report(valid_set.classes, y_pred))

# plot confusion matrix 

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
def plot_confusion_matrix(normalize):
  classes = ['normal','follicular', 'intense', 'scar', 'trichiasis','corneal opacity']
  tick_marks = [0.5,1.5]
  cn = confusion_matrix(valid_set, y_pred, normalize=normalize)
  sns.heatmap(cn,cmap='plasma',annot=True)
  plt.xticks(tick_marks, classes)
  plt.yticks(tick_marks, classes)
  plt.title('Confusion Matrix')
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

print('Confusion Matrix without Normalization')
plot_confusion_matrix(normalize=None)

print('Confusion Matrix with Normalized Values')
plot_confusion_matrix(normalize='true')

# visualization of accuracy/loss

In [ ]:
N= 100
plt.style.use("ggplot")
plt.figure()
plt.plot( r.history["loss"], label="train loss")
plt.plot( r.history[" val_loss"], label="val loss")
plt.plot( r.history["accuracy"], label="train accuracy")
plt.plot(r.history["val_accuracy"], label="val_accuracy")
plt.title("training loss and Accuracy)
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("drive/My Drive/Vgg16")